# Intro/Imports
This notebook pulls together pieces to complete a cross validation analysis of the two tiered classification method. The end result is a 93% classification accuracy.

In [1]:
import pickle
import os
import numpy as np
import sys
sys.path.append('../..')
import OtolithAnalysis as oa
import tensorflow as tf
from tensorflow import keras

In [2]:
nim_fold = 15

In [3]:
# This order was printed and copied from the AWS training script
# where it was generated randomly (with a seed of 1)
# cvorder = [14,98,75,16,131,56,141,44,29,120,94,5,102,51,78,42,
#            92,66,31,35,90,84,77,40,125,99,33,19,73,146,91,135,
#            69,128,114,48,53,28,54,108,112,17,119,103,58,118,18,
#            4,45,59,39,36,117,139,107,132,126,85,122,95,11,113,
#            123,12,2,104,6,127,110,65,55,144,138,46,62,74,116,
#            93,100,89,10,34,32,124,38,83,111,149,27,23,67,9,130,
#            97,105,145,87,148,109,64,15,82,41,80,52,26,76,43,24,
#            136,121,143,49,21,70,3,142,30,147,106,47,115,13,88,
#            8,81,60,0,1,57,22,61,63,7,86,96,68,50,101,20,25,134,
#            71,129,79,133,137,72,140,37]
# pickle.dump(cvorder,open('crossval_order.p', 'wb'))

In [3]:
dir0 = 'ClassifierSampleSelection'
subdir1 = 'Samples'
subdir2 = 'Scores'
subdir3 = 'Score_Tables'
dir1 = 'ClassifierSampleSelection50'
pstr = '2019_08_18_ClassifierNetworkForSelection_'
destdir = 'ClassifierSampleSelectionCombined'
# os.mkdir(destdir)
# os.mkdir(os.path.join(destdir, subdir1))
# os.mkdir(os.path.join(destdir, subdir2))
# os.mkdir(os.path.join(destdir, subdir3))

In [24]:
# def combine_lists(ftype, subdir, ind):
#     with open(os.path.join(dir0, subdir, pstr + 
#                        ftype + '_' + str(ind) + '.p'), 'rb') as f:
#         samp = pickle.load(f)
#     with open(dir1 + '/' + pstr + ftype + '50_' + str(ind) + '.p', 'rb') as f:
#         samp50 = pickle.load(f)
#     samp.extend(samp50)
#     with open(os.path.join(destdir, subdir, ftype + '_' + str(ind) + '.p'), 'wb') as f:
#         pickle.dump(samp, f)
# for ind in range(10):
#     combine_lists('SmoothedSamples', subdir1, ind)
#     combine_lists('Scores', subdir2, ind)
#     combine_lists('ScoreTable', subdir3, ind)

In [4]:
def make_transfer_module(model):
    model2 = keras.Sequential()
    model2.add(keras.layers.Conv1D(256, 3, weights=model.layers[0].get_weights(), activation=tf.nn.relu))
    model2.add(keras.layers.Conv1D(128, 9, activation=tf.nn.relu, weights=model.layers[1].get_weights()))
    model2.add(keras.layers.Flatten())
    model2.add(keras.layers.Dense(256, activation=tf.nn.relu, weights=model.layers[3].get_weights()))
    return model2

def make_none_marked_model():
    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation=tf.nn.relu))
    model.add(keras.layers.Dense(2, activation=tf.nn.softmax))
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [5]:
with open('crossval_order.p', 'rb') as f:
    cvorder = np.array(pickle.load(f))

In [48]:
# extra_none = []
# for im_ind in range(30):
#     if im_ind % 5 == 0:
#         print("Currently evaluating image number {}".format(im_ind))
#     for s_name in os.listdir(none_fbase + '/' + str(im_ind)):
#         extra_none.append([])
#         with open(os.path.join(none_fbase, str(im_ind), s_name), 'rb') as f:
#             s = pickle.load(f)
#         extra_none[-1].append(oa.finder_network.fcn_smoothed_d2ydx2(s))
#     extra_none[-1] = np.array(extra_none[-1])
# extra_none = np.array(extra_none)

Currently evaluating image number 0
Currently evaluating image number 5
Currently evaluating image number 10
Currently evaluating image number 15
Currently evaluating image number 20
Currently evaluating image number 25


In [6]:
def make_train_array(fold_ind, nim_fold):
    # Generates an array of samples for training from within the training set. Recall that in this case,
    # the order of samples loaded in the with statement below follows cvorder.
    with open(os.path.join(destdir, subdir1, 
                       'SmoothedSamples_' + str(fold_ind) 
                       + '.p') , 'rb') as f:
        train_array = pickle.load(f)
    tr = []
    tr_labs = []
    for t_ind in range(len(train_array)):
        if t_ind not in range(fold_ind * nim_fold, (fold_ind + 1) * nim_fold):
            tr.extend(train_array[t_ind])
            mark_ind, im_ind = fcn_mark_im_ind(cvorder[t_ind], 30)
            if mark_ind == 2:
                lab = 0
            else:
                lab = 1
            tr_labs.extend([lab for _ in range(len(train_array[t_ind]))])
    return tr, tr_labs, train_array

def fcn_mark_im_ind(im_ind_raw, n_img):
    """
    Calculates the mark an image index based on the total image index
    For example, im_ind_raw is 52 and n_img is 30, then mark_ind is 1
    and im_ind is 22
    """
    mark_ind = int(im_ind_raw/n_img)
    im_ind = im_ind_raw - mark_ind * n_img
    return mark_ind, im_ind

In [9]:
y = []
yhat = []
save_scores = []
fmod_name_base = 'BinaryNetworks/binarynet_'
for fold_ind in range(10):
    print("Currently evaluating fold number {}".format(fold_ind))
    mod = keras.models.load_model(fmod_name_base + str(fold_ind) + '.h5', compile=False)
    temp_model = make_transfer_module(mod)
    tr, tr_labs, train_array = make_train_array(fold_ind, nim_fold)
    while np.mean(tr_labs) > 0.5:
        none_im_ind = np.random.randint(0, 30)
        while none_im_ind + 60 in cvorder[fold_ind * nim_fold:(fold_ind + 1) * nim_fold]:
            none_im_ind = np.random.randint(0, 30)
        tr_ind = np.where(cvorder==none_im_ind + 60)[0][0]
        tr.extend(train_array[tr_ind])
        tr_labs.extend([ 0 for _ in range(len(train_array[tr_ind]))])
    tr = np.array(tr)
    tr_labs = np.array(tr_labs, dtype=int)
    tr_2 = temp_model.predict(np.expand_dims(tr, axis=2))
    nm_model = make_none_marked_model()
    nm_model.fit(np.expand_dims(tr_2, axis=2), tr_labs, epochs=20)
    print('check 1')
    for ind in range(fold_ind * nim_fold, (fold_ind + 1) * nim_fold):
        samps = train_array[ind]
        samps_2 = temp_model.predict(np.expand_dims(samps, axis=2))
        mark_ind, im_ind = fcn_mark_im_ind(cvorder[ind], 30)
        if mark_ind == 2:
            y.append(0)
        else:
            y.append(1)
        scores = np.mean(nm_model.predict(samps_2), axis=0)
        save_scores.append(scores)

Currently evaluating fold number 0
Epoch 1/20
3450/3450 [==============================] - 0s 43us/sample - loss: 0.4835 - acc: 0.8374
Epoch 2/20
3450/3450 [==============================] - 0s 30us/sample - loss: 0.2368 - acc: 0.9232
Epoch 3/20
3450/3450 [==============================] - 0s 29us/sample - loss: 0.1984 - acc: 0.9368
Epoch 4/20
3450/3450 [==============================] - 0s 29us/sample - loss: 0.1787 - acc: 0.9414
Epoch 5/20
3450/3450 [==============================] - 0s 29us/sample - loss: 0.1634 - acc: 0.9441
Epoch 6/20
3450/3450 [==============================] - 0s 31us/sample - loss: 0.1518 - acc: 0.9472
Epoch 7/20
3450/3450 [==============================] - 0s 32us/sample - loss: 0.1413 - acc: 0.9510
Epoch 8/20
3450/3450 [==============================] - 0s 30us/sample - loss: 0.1315 - acc: 0.9533
Epoch 9/20
3450/3450 [==============================] - 0s 31us/sample - loss: 0.1238 - acc: 0.9559
Epoch 10/20
3450/3450 [==============================] - 0s 33us/

3911/3911 [==============================] - 0s 32us/sample - loss: 0.0050 - acc: 0.9992
Epoch 20/20
3911/3911 [==============================] - 0s 32us/sample - loss: 0.0046 - acc: 0.9992
check 1
Currently evaluating fold number 8
Epoch 1/20
5367/5367 [==============================] - 0s 45us/sample - loss: 0.5088 - acc: 0.8127
Epoch 2/20
5367/5367 [==============================] - 0s 28us/sample - loss: 0.2127 - acc: 0.9324
Epoch 3/20
5367/5367 [==============================] - 0s 30us/sample - loss: 0.1562 - acc: 0.9503
Epoch 4/20
5367/5367 [==============================] - 0s 31us/sample - loss: 0.1272 - acc: 0.9592
Epoch 5/20
5367/5367 [==============================] - 0s 31us/sample - loss: 0.1062 - acc: 0.9646
Epoch 6/20
5367/5367 [==============================] - 0s 32us/sample - loss: 0.0903 - acc: 0.9702
Epoch 7/20
5367/5367 [==============================] - 0s 31us/sample - loss: 0.0788 - acc: 0.9735
Epoch 8/20
5367/5367 [==============================] - 0s 31us/sam

In [8]:
type(samps)

numpy.ndarray

In [57]:
yhat = []
for s in save_scores:
    if s[0] > 0.29:
        yhat.append(0)
    else:
        yhat.append(1)
acc = np.sum(np.array(yhat)==np.array(y))/len(yhat)
print('Binary accuracy at expected optimal cutoff: {:0.3f}'.format(acc))

Binary accuracy at expected optimal cutoff: 0.947


In [63]:
scbase = 'ClassifierSampleSelectionCombined/Scores/Scores_'
yhatclass = np.ones(150, dtype=int) * -1
yhatfin = np.ones(150, dtype=int) * 2
yfin = np.ones(150, dtype=int) * -1
for fold_ind in range(10):
    with open(scbase + str(fold_ind) + '.p', 'rb') as f:
        sc = pickle.load(f)
    for ind in range(nim_fold * fold_ind, (fold_ind + 1) * nim_fold):
        mark, im_ind = fcn_mark_im_ind(cvorder[ind], 30)
        sctemp = np.average(sc[ind], axis=0)
        yhatclass[ind] = np.argmax(sctemp)
        if yhat[ind] != 0:
            yhatfin[ind] = yhatclass[ind] 
        yfin[ind] = mark
print('Overall classification accuracy: {:0.3f}'.format(np.sum(yhatfin==yfin) / len(yfin)))

Overall classification accuracy: 0.927


In [70]:
cond = yfin != 2
print('marked classification accuracy: {:0.3f}'.
      format(np.sum(yfin[cond] == yhatfin[cond]) / np.sum(cond)))
icond = np.invert(cond)
print('Unmarked classification accuracy: {:0.3f}'.
      format(np.sum(yfin[icond] == yhatfin[icond]) / np.sum(icond)))
print("Marked classification accuracy before binary filter: {:0.3f}".
     format(np.sum(yhatclass[cond] == yfin[cond]) / np.sum(cond)))
print("Marked binary accuracy: {:0.3f}".
     format(np.sum(np.array(yhat)[cond] == 1) / np.sum(cond)))

marked classification accuracy: 0.917
Unmarked classification accuracy: 0.967
Marked classification accuracy before binary filter: 0.950
Marked binary accuracy: 0.942


In [71]:
for cutoff in np.linspace(0.1, 0.9, 9):
    yhat = []
    for s in save_scores:
        if s[0] > cutoff:
            yhat.append(0)
        else:
            yhat.append(1)
    yhat = np.array(yhat)
    print("Cutoff: {:0.1f}, binary accuracy: {:0.3f}".format(cutoff, np.sum(yhat==y)/len(yhat)))

Cutoff: 0.1, binary accuracy: 0.753
Cutoff: 0.2, binary accuracy: 0.900
Cutoff: 0.3, binary accuracy: 0.953
Cutoff: 0.4, binary accuracy: 0.953
Cutoff: 0.5, binary accuracy: 0.920
Cutoff: 0.6, binary accuracy: 0.900
Cutoff: 0.7, binary accuracy: 0.873
Cutoff: 0.8, binary accuracy: 0.833
Cutoff: 0.9, binary accuracy: 0.813


In [10]:
np.random.binomial(100, 0.927)

93

In [12]:
res = []
for i in range(10000):
    res.append(np.random.binomial(100, 0.927))
print(np.sum(np.array(res) <= 89) / 10000)

0.1159
